In [ ]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown, clear_output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as mtick
import gdown

warnings.filterwarnings('ignore')

# Styling untuk HTML
header_style = """
<style>
.header {
    background-color: #4e73df;
    background-image: linear-gradient(180deg, #4e73df 10%, #224abe 100%);
    color: white;
    padding: 20px;
    border-radius: 10px;
    margin-bottom: 20px;
    text-align: center;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
}
.subheader {
    background-color: #36b9cc;
    color: white;
    padding: 15px;
    border-radius: 8px;
    margin-bottom: 15px;
    text-align: center;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
}
.dashboard-container {
    display: flex;
    flex-direction: column;
    gap: 20px;
    padding: 20px;
    max-width: 1200px;
    margin: 0 auto;
    background-color: #f8f9fc;
    border-radius: 10px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.05);
}
.card {
    background-color: white;
    border-radius: 8px;
    padding: 20px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    margin-bottom: 20px;
}
.insight-text {
    background-color: #f6f8fd;
    border-left: 4px solid #4e73df;
    padding: 15px;
    margin-top: 15px;
    border-radius: 0 8px 8px 0;
}
.btn {
    background-color: #4e73df;
    color: white;
    border: none;
    padding: 10px 20px;
    border-radius: 5px;
    cursor: pointer;
    font-weight: bold;
    transition: all 0.3s;
}
.btn:hover {
    background-color: #2e59d9;
    transform: translateY(-2px);
}
.loading {
    display: none; /* Sembunyikan loading indicator */
}
@keyframes spin {
    to { transform: rotate(360deg); }
}
</style>
"""

# Tampilkan header
display(HTML(header_style))
display(HTML("""
<div class="header">
    <h1>MIDTERM PROJECT - Data Visualization</h1>
</div>
"""))

display(HTML("""
<div class="insight-text">
<h3>Visualisasi Karir Developer Modern Tahun 2023</h3>
<p>Dalam bagian ini, akan dibahas <b>"Karir Developer Modern tahun 2023"</b> melalui 4 visualisasi data yang saling berkaitan,
masing-masing merepresentasikan aspek penting dalam perjalanan seorang developer :</p>
<ol>
    <li><b>Demografi Developer :</b> Visualisasi ini akan menampilkan gambaran umum tentang latar belakang para developer—usia, lokasi geografis, gender, dan lain-lain.</li>
    <li><b>Jalur Pendidikan :</b> Menyoroti jalur pendidikan formal dan informal yang umum diambil developer pada umur dengan jumlah terbanyak, seperti kuliah, bootcamp, atau belajar otodidak.</li>
    <li><b>Keterampilan dan Pengalaman :</b> Visualisasi ini fokus pada bahasa pemrograman, teknologi populer, serta pengalaman kerja para developer pada umur dengan jumlah terbanyak.</li>
    <li><b>Lingkungan Kerja :</b> Membahas model kerja (remote/hybrid/on-site), metode kerja (scrum, agile), dan tools yang digunakan para developer pada umur dengan jumlah terbanyak.</li>
</ol>
</div>
"""))

# Buat layout dashboard, tapi tanpa loading indicator
display(HTML("""
<div class="dashboard-container">
</div>
"""))

# Fungsi untuk menampilkan progress - sekarang tidak menampilkan apa-apa
def show_progress(text):
    pass  # Tidak menampilkan apa-apa

# -------------------------------------------------------------------------
# TAHAP 1: MEMBACA DAN EKSPLORASI DATA
# -------------------------------------------------------------------------
# Download dari Google Drive (ID file)
file_id = "1uj5KvqS3U91sF5Uy1EoTbNJiQXbNIm0k"
url = f"https://drive.google.com/uc?id={file_id}"
file_path = "survey_results_public_2023.csv"
gdown.download(url, output, quiet=True)

# Membaca dataset setelah didownload
df = pd.read_csv(file_path)

# Tampilkan informasi dataset
display(HTML("""
<div class="subheader">
    <h2>Informasi Dataset</h2>
</div>
"""))

# Buat widget tab untuk informasi dataset
tab = widgets.Tab()
tab_eksplorasi = widgets.Output()
tab_kolom = widgets.Output()
tab_null = widgets.Output()

# Tab 1: Informasi umum
with tab_eksplorasi:
    display(HTML(f"""
    <div class="card">
        <h3>Informasi Umum Dataset</h3>
        <p>Jumlah baris: {df.shape[0]:,}</p>
        <p>Jumlah kolom: {df.shape[1]}</p>
        <p>Memori yang digunakan: {df.memory_usage().sum() / 1024**2:.2f} MB</p>
    </div>
    <div class="card">
        <h3>Sampel Data</h3>
        {df.head(3).to_html(classes='table table-striped')}
    </div>
    """))

# Tab 2: Informasi kolom
with tab_kolom:
    # Buat DataFrame untuk informasi kolom
    col_info = pd.DataFrame({
        'Nama Kolom': df.columns,
        'Tipe Data': df.dtypes.astype(str),
        'Non-Null': df.count(),
        'Null (%)': (df.isnull().mean() * 100).round(2)
    })
    display(HTML(f"""
    <div class="card">
        <h3>Informasi Kolom</h3>
        {col_info.head(10).to_html(classes='table table-striped')}
        <p><i>Menampilkan 10 dari {len(col_info)} kolom</i></p>
    </div>
    """))

# Tab 3: Missing values
with tab_null:
    # Buat DataFrame untuk nilai null
    null_info = pd.DataFrame({
        'Kolom': df.isnull().sum().index,
        'Jumlah Null': df.isnull().sum().values,
        'Persentase (%)': (df.isnull().mean() * 100).values.round(2)
    })
    null_info = null_info.sort_values('Jumlah Null', ascending=False)
    
    display(HTML(f"""
    <div class="card">
        <h3>Missing Values</h3>
        {null_info.head(10).to_html(classes='table table-striped')}
        <p><i>Menampilkan 10 kolom dengan missing values terbanyak</i></p>
    </div>
    """))

# Gabungkan semua tab
tab.children = [tab_eksplorasi, tab_kolom, tab_null]
tab.set_title(0, 'Eksplorasi')
tab.set_title(1, 'Kolom')
tab.set_title(2, 'Missing Values')
display(tab)

# -------------------------------------------------------------------------
# TAHAP 2: PEMBERSIHAN DATA (tanpa menampilkan teks progress)
# -------------------------------------------------------------------------

# Pilih kolom-kolom yang akan digunakan
columns_to_use = ['Age', 'DevType', 'YearsCode', 'YearsCodePro', 'EdLevel', 'RemoteWork', 
                  'LanguageHaveWorkedWith', 'ConvertedCompYearly', 'Country', 'MainBranch']
df_clean = df[columns_to_use].copy()

# Tangani missing values di kolom-kolom kategorikal
categorical_cols = ['Age', 'DevType', 'EdLevel', 'RemoteWork', 'Country', 'MainBranch']
for col in categorical_cols:
    if col in df_clean.columns:
        df_clean[col].fillna("Unknown", inplace=True)

# Tangani missing values di kolom numerik dan konversi format
if 'YearsCode' in df_clean.columns:
    df_clean['YearsCode'] = df_clean['YearsCode'].replace('Less than 1 year', '0.5')
    df_clean['YearsCode'] = df_clean['YearsCode'].replace('More than 50 years', '50')
    df_clean['YearsCode'] = pd.to_numeric(df_clean['YearsCode'], errors='coerce')
    df_clean['YearsCode'].fillna(df_clean['YearsCode'].median(), inplace=True)

if 'YearsCodePro' in df_clean.columns:
    df_clean['YearsCodePro'] = df_clean['YearsCodePro'].replace('Less than 1 year', '0.5')
    df_clean['YearsCodePro'] = df_clean['YearsCodePro'].replace('More than 50 years', '50')
    df_clean['YearsCodePro'] = pd.to_numeric(df_clean['YearsCodePro'], errors='coerce')
    df_clean['YearsCodePro'].fillna(df_clean['YearsCodePro'].median(), inplace=True)

if 'ConvertedCompYearly' in df_clean.columns:
    # Isi missing values dengan median
    df_clean['ConvertedCompYearly'].fillna(df_clean['ConvertedCompYearly'].median(), inplace=True)
    
    # Tangani outliers pada kolom ConvertedCompYearly
    q1 = df_clean['ConvertedCompYearly'].quantile(0.25)
    q3 = df_clean['ConvertedCompYearly'].quantile(0.75)
    iqr = q3 - q1
    upper_bound = q3 + 1.5 * iqr
    df_clean.loc[df_clean['ConvertedCompYearly'] > upper_bound, 'ConvertedCompYearly'] = upper_bound

# Pastikan kolom Age memiliki urutan yang benar
if 'Age' in df_clean.columns:
    # Definisikan urutan usia yang benar
    age_order = ['Under 18 years old', '18-24 years old', '25-34 years old', 
                 '35-44 years old', '45-54 years old', '55-64 years old', 
                 '65 years or older', 'Unknown']
    
    # Ubah tipe data menjadi categorical dengan urutan yang benar
    df_clean['Age'] = pd.Categorical(df_clean['Age'], categories=age_order, ordered=True)

# -------------------------------------------------------------------------
# TAHAP 3: FILTER DATA UNTUK KELOMPOK USIA 25-34 TAHUN
# -------------------------------------------------------------------------

# Filter dataset untuk fokus pada kelompok usia 25-34 tahun
df_25_34 = df_clean[df_clean['Age'] == '25-34 years old'].copy()

# Tampilkan ringkasan filter
total_developers = len(df_clean)
young_developers = len(df_25_34)
percentage = young_developers / total_developers * 100

display(HTML(f"""
<div class="card">
    <h3>Ringkasan Filter Data</h3>
    <div style="display: flex; justify-content: space-around; text-align: center; margin-bottom: 20px;">
        <div style="background-color: #f8f9fc; padding: 15px; border-radius: 8px; width: 30%;">
            <h4 style="color: #4e73df;">Total Developer</h4>
            <p style="font-size: 24px; font-weight: bold;">{total_developers:,}</p>
        </div>
        <div style="background-color: #f8f9fc; padding: 15px; border-radius: 8px; width: 30%;">
            <h4 style="color: #1cc88a;">Developer 25-34 Tahun</h4>
            <p style="font-size: 24px; font-weight: bold;">{young_developers:,}</p>
        </div>
        <div style="background-color: #f8f9fc; padding: 15px; border-radius: 8px; width: 30%;">
            <h4 style="color: #36b9cc;">Persentase</h4>
            <p style="font-size: 24px; font-weight: bold;">{percentage:.1f}%</p>
        </div>
    </div>
</div>
"""))

# -------------------------------------------------------------------------
# VISUALISASI DATA INTERAKTIF
# -------------------------------------------------------------------------
display(HTML("""
<div class="subheader">
    <h2>Visualisasi Data Interaktif</h2>
</div>
"""))

# Buat tab untuk setiap visualisasi
vis_tabs = widgets.Tab()
tab_vis1 = widgets.Output()
tab_vis2 = widgets.Output()
tab_vis3 = widgets.Output()
tab_vis4 = widgets.Output()

# -------------------------------------------------------------------------
# VISUALISASI 1: DISTRIBUSI USIA DEVELOPER (PLOTLY)
# -------------------------------------------------------------------------
with tab_vis1:
    if 'Age' in df_clean.columns:
        # Hitung frekuensi setiap kategori
        age_counts = df_clean['Age'].value_counts().reindex(age_order).reset_index()
        age_counts.columns = ['Age', 'Count']
        age_counts['Percentage'] = age_counts['Count'] / age_counts['Count'].sum() * 100
        
        # Buat warna untuk highlight 25-34 tahun
        colors = ['rgba(55, 83, 109, 0.7)'] * len(age_counts)
        for i, age in enumerate(age_counts['Age']):
            if age == '25-34 years old':
                colors[i] = 'rgba(246, 78, 139, 0.9)'
        
        # Buat visualisasi interaktif dengan Plotly
        fig = px.bar(
            age_counts, 
            x='Age', 
            y='Count',
            text=age_counts['Percentage'].apply(lambda x: f'{x:.1f}%'),
            title='Distribusi Usia Developer',
            labels={'Count': 'Jumlah Responden', 'Age': 'Kelompok Usia'},
            height=500
        )
        
        fig.update_traces(
            marker_color=colors,
            textposition='outside'
        )
        
        # Perbaiki layout untuk mencegah tumpang tindih text
        fig.update_layout(
            title_font_size=20,
            xaxis_title_font_size=14,
            yaxis_title_font_size=14,
            template='plotly_white',
            xaxis_tickangle=-45,
            margin=dict(l=50, r=50, t=80, b=100),  # Tambahkan margin bawah
            uniformtext_minsize=10,
            uniformtext_mode='hide'
        )
        
        fig.show()
        
        # Tampilkan insight
        display(HTML("""
        <div class="insight-text">
            <h4>Insight:</h4>
            <p>Developer saat ini didominasi oleh kelompok usia 25-34 tahun, menunjukkan bahwa 
            industri pengembangan software terus menarik talenta di usia produktif. Visualisasi 
            selanjutnya akan fokus pada kelompok usia dominan ini untuk memahami karakteristik
            dan preferensi mereka secara lebih mendalam.</p>
        </div>
        """))
    else:
        display(HTML("<p>Kolom Age tidak tersedia untuk visualisasi</p>"))

# -------------------------------------------------------------------------
# VISUALISASI 2: JALUR PENDIDIKAN DEVELOPER USIA 25-34 TAHUN (PLOTLY)
# -------------------------------------------------------------------------
with tab_vis2:
    if 'EdLevel' in df_25_34.columns:
        # Menghitung distribusi tingkat pendidikan
        ed_counts = df_25_34['EdLevel'].value_counts()
        
        # Menghapus 'Unknown' jika terlalu banyak
        if 'Unknown' in ed_counts.index and ed_counts['Unknown'] > ed_counts.sum() * 0.2:
            ed_counts = ed_counts.drop('Unknown')
        
        # Ambil 6 tingkat pendidikan teratas
        ed_counts = ed_counts.head(6)
        
        if len(ed_counts) > 0:
            # Konversi ke DataFrame untuk Plotly
            ed_df = pd.DataFrame({'EdLevel': ed_counts.index, 'Count': ed_counts.values})
            ed_df['Percentage'] = ed_df['Count'] / ed_df['Count'].sum() * 100
            ed_df = ed_df.sort_values('Count')  # Urutkan untuk visualisasi yang lebih baik
            
            # Buat visualisasi interaktif dengan Plotly
            fig = px.bar(
                ed_df,
                x='Count',
                y='EdLevel',
                text=ed_df['Percentage'].apply(lambda x: f'{x:.1f}%'),
                title='Jalur Pendidikan Developer Usia 25-34 Tahun',
                labels={'Count': 'Jumlah Responden', 'EdLevel': 'Tingkat Pendidikan'},
                color='Count',
                color_continuous_scale='RdBu',
                height=500
            )
            
            # Perbaiki tampilan teks untuk mencegah tumpang tindih
            fig.update_traces(
                textposition='outside',
                textfont=dict(size=12)
            )
            
            fig.update_layout(
                title_font_size=20,
                xaxis_title_font_size=14,
                yaxis_title_font_size=14,
                template='plotly_white',
                margin=dict(l=50, r=50, t=80, b=50),
                yaxis=dict(tickfont=dict(size=12))  # Perbaiki ukuran font pada sumbu y
            )
            
            fig.show()
            
            # Ambil top education
            top_edu = ed_df.iloc[-1]['EdLevel']
            top_pct = ed_df.iloc[-1]['Percentage']
            second_edu = ed_df.iloc[-2]['EdLevel']
            second_pct = ed_df.iloc[-2]['Percentage']
            
            # Tampilkan insight
            display(HTML(f"""
            <div class="insight-text">
                <h4>Insight:</h4>
                <p>Developer usia 25-34 tahun dominan memiliki latar belakang pendidikan <b>{top_edu}</b> ({top_pct:.1f}%),
                diikuti oleh <b>{second_edu}</b> ({second_pct:.1f}%). Pola ini menunjukkan bahwa mayoritas developer di 
                usia produktif ini memiliki pendidikan formal tinggi, menggambarkan kebutuhan fondasi akademis yang kuat
                dalam industri pengembangan software modern.</p>
            </div>
            """))
        else:
            display(HTML("<p>Tidak ada data valid untuk tingkat pendidikan setelah pembersihan</p>"))
    else:
        display(HTML("<p>Kolom EdLevel tidak tersedia untuk visualisasi</p>"))

# -------------------------------------------------------------------------
# VISUALISASI 3: KETERAMPILAN DAN PENGALAMAN DEVELOPER USIA 25-34 TAHUN (PLOTLY)
# -------------------------------------------------------------------------
with tab_vis3:
    if 'LanguageHaveWorkedWith' in df_25_34.columns and 'YearsCode' in df_25_34.columns:
        # Buat subplot 1x2
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=('10 Bahasa Pemrograman Teratas (Developer 25-34 Tahun)', 
                           'Distribusi Pengalaman Developer (25-34 Tahun)'),
            specs=[[{"type": "bar"}, {"type": "bar"}]],
            horizontal_spacing=0.15  # Tambahkan ruang antara subplot
        )
        
        # Subplot 1: Bahasa Pemrograman
        lang_data = df_25_34['LanguageHaveWorkedWith'].dropna()
        
        if len(lang_data) > 0:
            # Pisahkan bahasa pemrograman
            all_langs = []
            for langs in lang_data:
                if isinstance(langs, str):
                    all_langs.extend(langs.split(';'))
            
            # Hitung frekuensi dan ambil 10 teratas
            lang_counts = pd.Series(all_langs).value_counts().head(10)
            lang_df = pd.DataFrame({'Language': lang_counts.index, 'Count': lang_counts.values})
            lang_df = lang_df.sort_values('Count')  # Urutkan untuk visualisasi
            
            # Tambahkan bar chart bahasa pemrograman
            fig.add_trace(
                go.Bar(
                    x=lang_df['Count'],
                    y=lang_df['Language'],
                    orientation='h',
                    marker=dict(color='rgba(55, 83, 109, 0.8)', line=dict(color='rgba(55, 83, 109, 1.0)', width=1)),
                    text=lang_df['Count'],
                    textposition='outside'
                ),
                row=1, col=1
            )
            
            # Subplot 2: Pengalaman
            years_data = df_25_34['YearsCode'].copy()
            
            # Buat kategori pengalaman
            bins = [0, 2, 5, 10, 15, 50]
            labels = ['0-2', '3-5', '6-10', '11-15', '16+']
            df_25_34['ExpCategory'] = pd.cut(years_data, bins=bins, labels=labels)
            exp_counts = df_25_34['ExpCategory'].value_counts().sort_index()
            
            # Buat DataFrame untuk visualisasi
            exp_df = pd.DataFrame({'Experience': exp_counts.index, 'Count': exp_counts.values})
            
            # Tambahkan bar chart pengalaman
            fig.add_trace(
                go.Bar(
                    x=exp_df['Experience'],
                    y=exp_df['Count'],
                    marker=dict(color='rgba(246, 78, 139, 0.8)', line=dict(color='rgba(246, 78, 139, 1.0)', width=1)),
                    text=exp_df['Count'],
                    textposition='auto'
                ),
                row=1, col=2
            )
            
            # Update layout
            fig.update_layout(
                height=550,  # Sedikit lebih tinggi untuk menghindari tumpang tindih
                template='plotly_white',
                showlegend=False,
                title_font=dict(size=20),
                margin=dict(l=50, r=50, t=100, b=50)  # Tambahkan margin atas
            )
            
            # Update axes dengan perbaikan layout
            fig.update_xaxes(title_text="Jumlah Developer", row=1, col=1, range=[0, max(lang_df['Count'])*1.15])  # Tambahkan ruang untuk teks
            fig.update_yaxes(title_text="Bahasa Pemrograman", row=1, col=1, tickfont=dict(size=11))  # Kurangi ukuran font
            fig.update_xaxes(title_text="Tahun Pengalaman", row=1, col=2)
            fig.update_yaxes(title_text="Jumlah Developer", row=1, col=2)
            
            fig.show()
            
            # Tampilkan insight
            top_lang = lang_df.iloc[-1]['Language']
            second_lang = lang_df.iloc[-2]['Language']
            third_lang = lang_df.iloc[-3]['Language']
            
            exp_mode = exp_df.loc[exp_df['Count'].idxmax(), 'Experience']
            exp_pct = exp_df.loc[exp_df['Count'].idxmax(), 'Count'] / exp_df['Count'].sum() * 100
            
            display(HTML(f"""
            <div class="insight-text">
                <h4>Insight:</h4>
                <p>Developer usia 25-34 tahun paling banyak menggunakan <b>{top_lang}</b>, <b>{second_lang}</b>, dan <b>{third_lang}</b>.
                Mayoritas dari mereka memiliki pengalaman <b>{exp_mode} tahun</b> ({exp_pct:.1f}%), yang sesuai dengan rentang usia 
                mereka dan menunjukkan bahwa banyak dari mereka yang memulai karir pengembangan software di awal atau 
                pertengahan 20-an.</p>
            </div>
            """))
        else:
            display(HTML("<p>Tidak ada data bahasa pemrograman yang valid</p>"))
    else:
        display(HTML("<p>Kolom LanguageHaveWorkedWith atau YearsCode tidak tersedia untuk visualisasi</p>"))

# -------------------------------------------------------------------------
# VISUALISASI 4: LINGKUNGAN KERJA DEVELOPER USIA 25-34 TAHUN (PLOTLY)
# -------------------------------------------------------------------------
with tab_vis4:
    if 'RemoteWork' in df_25_34.columns:
        # Menghitung distribusi pola kerja
        remote_counts = df_25_34['RemoteWork'].value_counts()
        
        # Menghapus 'Unknown' jika ada
        if 'Unknown' in remote_counts.index:
            remote_counts = remote_counts.drop('Unknown')
        
        if len(remote_counts) > 0:
            # Buat DataFrame untuk Plotly
            remote_df = pd.DataFrame({'RemoteWork': remote_counts.index, 'Count': remote_counts.values})
            remote_df['Percentage'] = remote_df['Count'] / remote_df['Count'].sum() * 100
            
            # Buat subplot 1x2
            fig = make_subplots(
                rows=1, cols=2,
                subplot_titles=('Proporsi Pola Kerja Developer (25-34 Tahun)', 
                               'Perbandingan Pola Kerja (25-34 Tahun)'),
                specs=[[{"type": "pie"}, {"type": "bar"}]],
                horizontal_spacing=0.15  # Tambahkan ruang antara subplot
            )
            
            # Subplot 1: Pie chart
            fig.add_trace(
                go.Pie(
                    labels=remote_df['RemoteWork'],
                    values=remote_df['Count'],
                    textinfo='label+percent',
                    hole=0.3,
                    marker=dict(colors=px.colors.sequential.Plasma_r),
                    pull=[0.05] * len(remote_df),
                    textfont=dict(size=12)  # Ukuran font yang lebih kecil untuk pie chart
                ),
                row=1, col=1
            )
            
            # Subplot 2: Bar chart
            fig.add_trace(
                go.Bar(
                    x=remote_df['RemoteWork'],
                    y=remote_df['Count'],
                    text=remote_df['Count'],
                    textposition='auto',
                    marker=dict(color=px.colors.sequential.Plasma_r)
                ),
                row=1, col=2
            )
            
            # Update layout dengan perbaikan untuk menghindari tumpang tindih
            fig.update_layout(
                height=550,
                template='plotly_white',
                showlegend=False,
                title_font=dict(size=20),
                margin=dict(l=50, r=50, t=100, b=100)  # Tambahkan margin atas dan bawah
            )
            
            # Update axes for bar chart
            fig.update_xaxes(
                title_text="Pola Kerja", 
                row=1, 
                col=2,
                tickangle=-30,  # Miringkan teks sumbu x
                tickfont=dict(size=10)  # Ukuran font yang lebih kecil
            )
            fig.update_yaxes(title_text="Jumlah Developer", row=1, col=2)
            
            fig.show()
            
            # Identifikasi pola dominan
            dominant_pattern = remote_df.loc[remote_df['Count'].idxmax(), 'RemoteWork']
            dominant_pct = remote_df.loc[remote_df['Count'].idxmax(), 'Percentage']
            
            # Tampilkan insight berdasarkan pola dominan
            insight_text = f"""
            <div class="insight-text">
                <h4>Insight:</h4>
                <p>Developer usia 25-34 tahun dominan bekerja dengan pola <b>{dominant_pattern}</b> ({dominant_pct:.1f}%),
            """
            
            if 'Remote' in dominant_pattern:
                insight_text += """
                menunjukkan bahwa generasi ini sangat mengadopsi model kerja jarak jauh.
                Ini mencerminkan preferensi mereka untuk fleksibilitas dan keseimbangan kehidupan-kerja.</p>
                """
            elif 'Hybrid' in dominant_pattern:
                insight_text += """
                menunjukkan bahwa mereka menghargai keseimbangan antara kolaborasi tatap muka dan fleksibilitas.
                Model hybrid ini sesuai dengan kebutuhan developer di usia produktif untuk networking dan mentoring.</p>
                """
            else:
                insight_text += """
                menunjukkan bahwa mereka masih menghargai interaksi langsung di lingkungan kantor tradisional.
                Ini mungkin mencerminkan kebutuhan kolaborasi dan mentoring dalam tahap pengembangan karir mereka.</p>
                """
            
            insight_text += "</div>"
            display(HTML(insight_text))
        else:
            display(HTML("<p>Tidak ada data valid untuk pola kerja setelah pembersihan</p>"))
    else:
        display(HTML("<p>Kolom RemoteWork tidak tersedia untuk visualisasi</p>"))

# Gabungkan semua tab visualisasi
vis_tabs.children = [tab_vis1, tab_vis2, tab_vis3, tab_vis4]
vis_tabs.set_title(0, 'Distribusi Usia')
vis_tabs.set_title(1, 'Jalur Pendidikan')
vis_tabs.set_title(2, 'Keterampilan & Pengalaman')
vis_tabs.set_title(3, 'Lingkungan Kerja')
display(vis_tabs)

# -------------------------------------------------------------------------
# KESIMPULAN INTERAKTIF
# -------------------------------------------------------------------------
show_progress("Membuat visualisasi lingkungan kerja...")
time.sleep(0.5)  # Simulasi loading

display(HTML("""
<div class="subheader">
    <h2>Kesimpulan</h2>
</div>
"""))

# Buat ringkasan yang lebih interaktif
display(HTML("""
<div class="card">
    <h3>Profil Developer Usia 25-34 Tahun</h3>
    
    <div id="summary-accordion">
        <!-- Panel 1 - Demografi -->
        <div class="accordion-item">
            <button class="btn" onclick="toggleAccordion('demografi')" style="width: 100%; margin-bottom: 5px; text-align: left; background-color: #4e73df;">
                1. Demografi ▼
            </button>
            <div id="demografi" class="accordion-content" style="display: block; padding: 10px; border-left: 3px solid #4e73df; margin-bottom: 15px;">
                <p>Kelompok usia 25-34 tahun mendominasi industri pengembangan software,
                menunjukkan bahwa profesi ini menarik bagi talenta di usia produktif.</p>
                <p>Mereka mewakili hampir setengah dari total responden survey, menjadikan mereka
                kekuatan utama dalam pembentukan tren dan praktik industri.</p>
            </div>
        </div>
        
        <!-- Panel 2 - Pendidikan -->
        <div class="accordion-item">
            <button class="btn" onclick="toggleAccordion('pendidikan')" style="width: 100%; margin-bottom: 5px; text-align: left; background-color: #1cc88a;">
                2. Pendidikan ▼
            </button>
            <div id="pendidikan" class="accordion-content" style="display: none; padding: 10px; border-left: 3px solid #1cc88a; margin-bottom: 15px;">
                <p>Mayoritas memiliki gelar sarjana atau master, menggambarkan pentingnya
                fondasi akademis yang kuat dalam pengembangan software modern.</p>
                <p>Pendidikan formal tetap menjadi jalur utama untuk memasuki industri,
                meskipun jalur alternatif seperti bootcamp dan pembelajaran mandiri juga ada.</p>
            </div>
        </div>
        
        <!-- Panel 3 - Keterampilan -->
        <div class="accordion-item">
            <button class="btn" onclick="toggleAccordion('keterampilan')" style="width: 100%; margin-bottom: 5px; text-align: left; background-color: #36b9cc;">
                3. Keterampilan ▼
            </button>
            <div id="keterampilan" class="accordion-content" style="display: none; padding: 10px; border-left: 3px solid #36b9cc; margin-bottom: 15px;">
                <p>Mereka dominan menggunakan bahasa pemrograman modern seperti JavaScript,
                Python, dan HTML/CSS, dengan pengalaman yang sesuai dengan usia mereka.</p>
                <p>Sebagian besar memiliki 3-10 tahun pengalaman, menunjukkan bahwa mereka
                memulai karir di usia 20-an dan telah mengembangkan keahlian yang signifikan.</p>
            </div>
        </div>
        
        <!-- Panel 4 - Lingkungan Kerja -->
        <div class="accordion-item">
            <button class="btn" onclick="toggleAccordion('lingkungan')" style="width: 100%; margin-bottom: 5px; text-align: left; background-color: #f6c23e;">
                4. Lingkungan Kerja ▼
            </button>
            <div id="lingkungan" class="accordion-content" style="display: none; padding: 10px; border-left: 3px solid #f6c23e; margin-bottom: 15px;">
                <p>Mereka mengadopsi model kerja yang fleksibel (hybrid/remote),
                mencerminkan preferensi generasi ini untuk keseimbangan kehidupan-kerja.</p>
                <p>Transformasi ini mungkin dipercepat oleh pandemi, tetapi telah menjadi
                preferensi yang bertahan dan bahkan menjadi ekspektasi dalam industri.</p>
            </div>
        </div>
    </div>
</div>

<script>
function toggleAccordion(id) {
  var content = document.getElementById(id);
  if (content.style.display === "block") {
    content.style.display = "none";
  } else {
    content.style.display = "block";
  }
}
</script>
"""))

# Tambahkan catatan metodologi
display(HTML("""
<div class="card">
    <h3>Catatan Metodologi</h3>
    <p>Analisis ini menggunakan data dari Stack Overflow Survey 2023, dengan fokus pada 
    responden berusia 25-34 tahun. Proses analisis meliputi:</p>
    <ol>
        <li>Pembersihan data (menangani missing values, outlier, dan format data)</li>
        <li>Filtering untuk kelompok usia 25-34 tahun</li>
        <li>Visualisasi berbagai aspek demografi, pendidikan, keterampilan, dan pola kerja</li>
        <li>Analisis pola dan tren dalam data</li>
    </ol>
    <p>Semua visualisasi dibuat menggunakan Plotly dan dapat dieksplorasi secara interaktif. 
    Dashboard ini dibuat menggunakan kombinasi Python, Plotly, dan Voila.</p>
</div>
"""))

# Footer
display(HTML("""
<div style="text-align: center; margin-top: 40px; padding: 20px; background-color: #f8f9fc; border-radius: 10px;">
    <p>© 2023 Stack Overflow Survey Analysis</p>
    <p style="font-size: 12px;">Dibuat menggunakan Python, Plotly dan Voila</p>
</div>
"""))

# Tambahkan ikon Font Awesome untuk UI yang lebih baik
display(HTML("""
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">
"""))